#### Computation of the gradient

From the earlier discussion, we have the cost function of MPF to be

$$ K(\theta) \approx \frac{1}{|\mathcal{D}|}\sum_{y\in \mathcal{D}}\sum_{h=1}^{n}\exp\bigg\{\delta_h * (\theta y)_h - 1/4 * diag(\theta)_h\bigg\}$$

and the gradient of the cost function with respect to the $\theta$ matrix is

$$ \frac{\partial K(\theta)}{\partial \theta_{ij}} = \begin{cases}\delta_iy_jk_i+\delta_jy_ik_j & i \neq j\\ \left(\delta_iy_i-\frac{1}{4}\right)k_i & i = j\\ \end{cases}$$

where $k_h = \exp\bigg\{\delta_h * (\theta y)_h - 1/4 * diag(\theta)_h\bigg\}$. We shall now work out how to explicitly compute the gradients using Python.

We start by recalling some definitions:
- $s$ :  samples where each row is the number of samples and each columns represent a unit in the restricted boltzmann machine, say $n$.
- $\theta$ : the parameter matrix to be learnt which has a size of $n \times n$

With the energy matrix $E$, we can compute the $\delta_ik_i$ terms by $\delta * k$, following by we can obtain the $\delta_iy_jk_i$ terms by taking the dot product of $\delta * k$ tranpose and $s$, which we shall call this matrix $D'$ that looks like 

$$ D'_{ij} = \begin{cases}\delta_iy_jk_i & i \neq j\\ \delta_iy_ik_i & i = j\\ \end{cases}$$

we extract the diagonals as $C$ and we add the missing $0.25 * k_i$ term to it by subtracting 0.25 times of the sum of the rows of $k$ from $C$. To form the $d_iy_jk_i + \delta_jy_ik_j$ term we remove the diagonals of $D'$ and call it $D''$, following which added the transpose of $D''$ to itself. We get the desired gradient matrix by filling the empty diagonals of $D'' + D''^\top$ with $C$.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import theano
# import theano.tensor as T


import os
import timeit
from datetime import datetime
from mpfntutils import load_data

from numpy.linalg import norm


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

In [2]:
def unravelparam(theta, units = 16):
    """
    Restores a vector of parameters into matrix form.
    """
    W = theta.reshape(units, units)
    return W


def ravelparam(W):
    """
    Ravels the parameters into a vector.
    """
    theta = W.ravel()
    return theta

In [3]:
units = 16
U = np.random.normal(loc = 0, scale = 1/units, size = (units, units))
W = 0.5 * (U + U.T)

In [4]:
samples = load_data('16-50K.npy')

In [5]:
def Kcost(x, W, temperature = 1):
        """
        Returns the cost computed by using the diagonals as the bias.
        Inputs:
        - x: samples used to train W.
        - W: weights between the neurons of the Boltzmann Machine (BM).
        - n: number of neurons in the BM.
        - temperature: keep it as 1 until cost grows too big then raise temperature.
        """
        num_samples = x.shape[0]        
        num_units = x.shape[1]
        delta = 1/2 - x
        diag = np.diag(W)[:, None].T
        E = delta * np.dot(x, W) - .25 * diag
        
        cost = np.sum(np.exp(1/temperature * E)) / num_samples         
        k = np.exp(E)        
        D = np.dot((delta * k).T, x)         
        C = np.zeros((num_units,))         
        np.copyto(C, np.diag(D))                 
        np.fill_diagonal(D, 0)         
        C = C - .25 * np.sum(k, axis = 0)         
        D = D + D.T         
        np.fill_diagonal(D, C) 

        return cost, D/ num_samples

In [6]:
cost, Wgrad = Kcost(samples, W)

#### Computation of Numerical gradient
For the computation of the numerical gradient for sanity checking, it is not as straightword as our $W$ matrix here is diagonal, meaning for a $3 \times 3$ $W$ matrix toy example,

$$\begin{pmatrix}\theta_{11} & \theta_{12} & \theta_{13} \\ \theta_{21} & \theta_{22} & \theta_{23} \\\theta_{31} & \theta_{32} & \theta_{33}  \end{pmatrix}$$

the gradients of $\theta_{ij}$ and $\theta_{ji}$ are though of as the same variables and hence in the computation of the numerical gradient for the parameter $\theta_{ij}$ using the formula 

$$\frac{K(\theta+\epsilon) - K(\theta-\epsilon)}{2\epsilon}$$

we have to add epsilon to both $\theta_{ij}$ and $\theta_{ji}$ to get their gradients (which are the same value). For example, to find the numerical gradient of $\theta_{12}$ we add epsilon to both $\theta_{12}$ and $\theta_{21}$, i.e.

$$\theta + \epsilon_{12} = \begin{pmatrix}\theta_{11} & \theta_{12} + \epsilon & \theta_{13} \\ \theta_{21}+ \epsilon & \theta_{22} & \theta_{23} \\\theta_{31} & \theta_{32} & \theta_{33}  \end{pmatrix}$$

here $\epsilon_{ij}$ is used to denote a small increment to the $\theta_{ij}$ and $\theta_{ji}$ value, thus $\theta + \epsilon_{ij} = \theta + \epsilon_{ji}$. Therefore,

$$\frac{\partial K(\theta)}{\partial\theta_{ij}} = \frac{K(\theta+\epsilon_{ij}) - K(\theta-\epsilon_{ij})}{2\epsilon}$$

To get the $\theta + \epsilon_{ij}$ matrix, using transponse, raveling and unraveling of the parameters will do the trick.

In [7]:
def computeNumericalGradient(J,W):
    EPSILON = 0.0001    
    W = ravelparam(W) 
    numgrad = np.zeros(np.shape(W)) 
    num_para = W.shape[0] 
    
    for i in range(num_para):
        w = np.zeros(np.shape(W))
        w[i] = EPSILON
        e_W = unravelparam(w)
        if not (e_W == e_W.T).all():
            e_W = ravelparam(e_W + e_W.T)
        else:
            e_W = ravelparam(e_W)
        wp = unravelparam(W + e_W)
        wm = unravelparam(W - e_W)
        numgrad[i] = (J(wp) - J(wm)) / (2 * EPSILON)
       
    return unravelparam(numgrad)

In [8]:
numgrad = computeNumericalGradient(lambda x: Kcost(samples, x)[0], W)

#### Comparing of analytic and numerical gradient
After getting both your numerical gradient, we error difference should be on the order of $1^{-10}$, which means that you analytic gradient that you implemented above is correct.

In [9]:
diff = norm(numgrad-Wgrad)/norm(numgrad+Wgrad)
print (diff)

2.60852798632e-10
